In [1]:
from keras.models import load_model

In [3]:
gender_model = load_model('G:/gender_detection.model')
age_model = load_model('G:/age_detection1.model')

In [5]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import cvlib as cv

# load gender and age detection models


# define gender and age classes
genclasses = ['male', 'female']
ageclasses = ['0-13', '13-23', '24-34', '35-44', '44-54', '55-64', '65-75', '75-85', '86-100+']

# open webcam
webcam = cv2.VideoCapture(0)

# loop through frames
while webcam.isOpened():

    # read frame from webcam
    status, frame = webcam.read()

    # apply face detection
    face, confidence = cv.detect_face(frame)

    # loop through detected faces
    for idx, f in enumerate(face):

        # get corner points of face rectangle
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]

        # draw rectangle over face
        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

        # crop the detected face region
        face_crop = np.copy(frame[startY:endY, startX:endX])

        if face_crop.shape[0] < 10 or face_crop.shape[1] < 10:
            continue

        # preprocessing for gender detection model
        face_crop = cv2.resize(face_crop, (96, 96))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)

        # apply gender detection on face
        conf = gender_model.predict(face_crop)[0]
        conf2 = age_model.predict(face_crop)[0]
        # get label with max accuracy
        idx = np.argmax(conf)
        idx2 = np.argmax(conf2)
        label = genclasses[idx]
        label2 = ageclasses[idx2]
        Y = startY - 10 if startY - 10 > 10 else startY + 10

        # write label and confidence above face rectangle
        cv2.putText(frame, label, (startX, Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, label2, (startX + 100, Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # display output
    cv2.imshow("gender-age detection", frame)

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release resources
webcam.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 28ms/step
